In [127]:
from functools import partial
from pyscf import gto, scf, cc
import numpy as np
from jax import numpy as jnp
from jax import vmap

from jax import config
config.update("jax_enable_x64", True)

print = partial(print, flush=True)

a = 1.
nH = 6
atoms = ""
for i in range(nH):
    atoms += f"H {i*a} 0 0 \n"

mol = gto.M(atom=atoms, basis="sto6g", verbose=4)
mf = scf.RHF(mol).density_fit()
mf.kernel()

mycc = cc.CCSD(mf)
e = mycc.kernel()

print(mycc.e_tot)
print(mycc.e_corr)

System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#5074-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sun Oct  5 16:27:56 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 6
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.000000000000   0.000000000000   0.000000000000 AA    0.000000000000   0.000000000000   0.00

******** <class 'pyscf.df.df.DF'> ********
auxbasis = None
max_memory = 4000
ETB for H: l = 0, exps = 0.200224856 * 2^n , n = 0..8

WARN: Even tempered Gaussians are generated as DF auxbasis for  H

init E= -2.42992244139156
  HOMO = -0.204045735961534  LUMO = 0.13840111927303
cycle= 1 E= -3.14955212808499  delta_E= -0.72  |g|= 0.111  |ddm|= 1.71
  HOMO = -0.324221355929661  LUMO = 0.214034118577356
cycle= 2 E= -3.15774245178069  delta_E= -0.00819  |g|= 0.0385  |ddm|= 0.246
  HOMO = -0.330464366202014  LUMO = 0.232009176330946
cycle= 3 E= -3.1589679408949  delta_E= -0.00123  |g|= 0.00665  |ddm|= 0.101
  HOMO = -0.333957770303784  LUMO = 0.232352367960061
cycle= 4 E= -3.15900173729129  delta_E= -3.38e-05  |g|= 0.000168  |ddm|= 0.0212
  HOMO = -0.3338448227675  LUMO = 0.232284255609341
cycle= 5 E= -3.15900175306578  delta_E= -1.58e-08  |g|= 1.37e-05  |ddm|= 0.000402
  HOMO = -0.333861918151968  LUMO = 0.232293175564557
cycle= 6 E= -3.15900175322303  delta_E= -1.57e-10  |g|= 1.22e-07  |dd

In [128]:
options = {'n_eql': 4,
           'n_prop_steps': 10,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 1,
            'seed': 2,
            'walker_type': 'rhf',
            'trial': 'rhf',
            'dt':0.005,
            'free_projection':False,
            'ad_mode':None,
            'use_gpu': False,
            }

In [131]:
from pyscf.ci.cisd import CISD
from pyscf.cc.ccsd import CCSD
from pyscf.cc.uccsd import UCCSD
from ad_afqmc.lno.cc import LNOCCSD
from ad_afqmc.lno_afqmc import lno_maker, afqmc_maker, lnoafqmc_runner
from pyscf import ao2mo, mcscf, scf, lib
from ad_afqmc.lno.base import lno

frozen = None
thresh = 1e-4
eris = None
run_frg_list = None

mfcc = mf
if isinstance(mfcc, (CCSD, CISD)):
    full_cisd = True
    mf = mfcc._scf
else:
    full_cisd = False
    mf = mfcc

if isinstance(thresh, list):
    thresh_occ, thresh_vir = thresh
else:
    thresh_occ = thresh*10
    thresh_vir = thresh

lno_cc = LNOCCSD(mf, thresh=thresh, frozen=frozen)
lno_cc.thresh_occ = thresh_occ
lno_cc.thresh_vir = thresh_vir
lno_cc.lo_type = 'boys'
lno_cc.no_type = 'ie'
no_type = 'ie'
lno_cc.frag_lolist = '1o'
lno_cc.force_outcore_ao2mo = True

s1e = mf.get_ovlp()
lococc = lno_cc.get_lo(lo_type='boys') # localized active occ orbitals
# lococc,locvir = lno_maker.get_lo(lno_cc,lo_type) ### fix this for DF
if eris is None: eris = lno_cc.ao2mo()

frag_lolist = [[i] for i in range(lococc.shape[1])]
print(frag_lolist)
nfrag = len(frag_lolist)

frozen_mask = lno_cc.get_frozen_mask()
thresh_pno = [thresh_occ,thresh_vir]
print(f'# lno thresh {thresh_pno}')

if run_frg_list is None:
    run_frg_list = range(nfrag)

frag_nonvlist = None
if frag_nonvlist is None: frag_nonvlist = lno_cc.frag_nonvlist
if frag_nonvlist is None: frag_nonvlist = [[None,None]] * nfrag

eorb_cc1 = np.empty(nfrag,dtype='float64')
eorb_cc2 = np.empty(nfrag,dtype='float64')
    
from jax import random
seeds = random.randint(random.PRNGKey(options["seed"]),
                    shape=(len(run_frg_list),), minval=0, maxval=100*nfrag)

for ifrag in run_frg_list:
    print(f'\n########### running fragment {ifrag+1} ##########')

    fraglo = frag_lolist[ifrag]
    orbfragloc = lococc[:,fraglo]
    THRESH_INTERNAL = 1e-10
    frag_target_nocc, frag_target_nvir = frag_nonvlist[ifrag]
    frzfrag1, orbfrag1, can_orbfrag1 \
        = lno_maker.make_lno(
            lno_cc,orbfragloc,THRESH_INTERNAL,thresh_pno
            )
    frzfrag2, orbfrag2, can_orbfrag2 \
         = lno.make_fpno1(lno_cc, eris, orbfragloc, no_type,
                            THRESH_INTERNAL, thresh_pno,
                            frozen_mask=frozen_mask,
                            frag_target_nocc=None,
                            frag_target_nvir=None,
                            canonicalize=True)

    # mol = mf.mol
    # nocc = mol.nelectron // 2 
    # nao = mol.nao
    # actfrag = np.array([i for i in range(nao) if i not in frzfrag])
    # # frzocc = np.array([i for i in range(nocc) if i in frzfrag])
    # actocc = np.array([i for i in range(nocc) if i in actfrag])
    # actvir = np.array([i for i in range(nocc,nao) if i in actfrag])
    # nactocc = len(actocc)
    # nactocc = len(actocc)
    # nactvir = len(actvir)
    # prjlo = orbfragloc.T @ s1e @ orbfrag[:,actocc]

    # print(f'# active orbitals: {actfrag}')
    # print(f'# active occupied orbitals: {actocc}')
    # print(f'# active virtual orbitals: {actvir}')
    # print(f'# frozen orbitals: {frzfrag}')

    eorb_cc1[ifrag],_,_ = lno_maker.lno_cc_solver(
                        mf,orbfrag2,orbfragloc,frozen=frzfrag1)
    # eorb_cc2[ifrag],_,_ = lno_maker.lno_cc_solver(
    #                     mf,orbfrag2,orbfragloc,frozen=frzfrag2)
    _,(_,eorb_cc2[ifrag],_) = lno_cc.impurity_solve(
         mf,orbfrag2,orbfragloc,frozen=frzfrag1,eris=eris)


lo_type = boys


******** <class 'pyscf.lo.boys.Boys'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.01
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3


init_guess = atomic
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 11.383029437648  delta_f= 11.383  |g|= 5.66107  1 KF 3 Hx
macro= 2  f(x)= 11.113897391135  delta_f= -0.269132  |g|= 4.95338  1 KF 3 Hx
macro= 3  f(x)= 10.883815879638  delta_f= -0.230082  |g|= 4.30722  1 KF 3 Hx
macro= 4  f(x)= 10.69625414059  delta_f= -0.187562  |g|= 3.59902  1 KF 3 Hx
macro= 5  f(x)= 10.554113995389  delta_f= -0.14214  |g|= 2.83824  1 KF 3 Hx
macro= 6  f(x)= 10.45963267315  delta_f= -0.0944813  |g|= 2.03552  1 KF 3 Hx
macro= 7  f(x)= 10.414344090377  delta_f= -0.0452886  |g|= 1.20206  1 KF 3 Hx
macro= 8  f(x)= 10.410208679768  delta_f= -0.00413541  |g|= 0.349396  1 KF 2 Hx
macro= 9  f(x)= 10.410208677215  delta_f= -2.55281e-09  |g|= 0.000264284  1 KF 2 Hx
macro X = 9  f(x)= 10.410208677215  |g|= 0.000264284  18 intor 9 KF 25 Hx
Lov is saved to /tmp/uob4yq55
[[0], [1], [2]]
# lno thresh [0.001, 0.0001]

########### running fragment 1 ##########
Using DF integrals
Lov is saved to /tmp/a28319x7
Using D

In [132]:
print(eorb_cc1)
print(eorb_cc2)
print(sum(eorb_cc1))
print(sum(eorb_cc2))
print(eorb_cc1[0])

[-0.02836883 -0.03351452 -0.03351452]
[-0.02836883 -0.03351452 -0.03351452]
-0.09539786181704976
-0.09539786181704978
-0.028368831406909253


In [ ]:
[-0.0273794401668634,-0.0326514520515449,-0.0326514520645434,-0.0273794401476732]

In [33]:
mycc.e_corr

-0.13207422268668184

In [54]:
for thresh in [1e-4]:
    mfcc = LNOCCSD(mf, thresh=thresh, frozen=frozen).set(verbose=5)
    mfcc.thresh_occ = 1e-4
    mfcc.thresh_vir = thresh
    mfcc.lo_type = 'boys'
    mfcc.no_type = 'ie'
    mfcc.frag_lolist = '1o'
    mfcc.ccsd_t = False
    mfcc.force_outcore_ao2mo = True
    mfcc.kernel(canonicalize=False)
    ecc = mfcc.e_corr
    # ecc_pt2corrected = mfcc.e_corr_pt2corrected(mmp.e_corr)
    print(f'thresh = {thresh:.2e}  E_corr(CCSD) = {ecc:.8f}'
          f'rel = {ecc/mycc.e_corr*100:.3f}%  diff = {ecc-mycc.e_corr:.8f}')


******** <class 'ad_afqmc.lno.cc.ccsd.LNOCCSD'> ********
nocc = 4, nmo = 8
max_memory 4000 MB (current use 272 MB)
thresh_occ = 1.000000e-04
thresh_vir = 1.000000e-04
force_outcore_ao2mo = True
no_type = ie
Constructing LOs
lo_type = boys


******** <class 'pyscf.lo.boys.Boys'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.01
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
Set conv_tol_grad to 0.000316228
macro= 1  f(x)= 17.85967232466  delta_f= 17.8597  |g|= 8.34017  1 KF 3 Hx
macro= 2  f(x)= 17.276939975683  delta_f= -0.582732  |g|= 7.45872  1 KF 3 Hx
macro= 3  f(x)= 16.767226643467  delta_f= -0.509713  |g|= 6.61597  1 KF 3 Hx
macro= 4  f(x)= 16.338077021812  delta_f= -0.42915  |g|= 5.67942  1 KF 3 Hx
macro= 5  f(x)= 15.995901310859  delta_f= -0.342176  |g|= 4.66179  1 KF 3 Hx
macro= 6  

In [97]:
from pyscf.lib import logger
from functools import reduce
lno_cc = LNOCCSD(mf, thresh=thresh, frozen=frozen)
lno_cc.thresh_occ = thresh_occ
lno_cc.thresh_vir = thresh_vir
lno_cc.lo_type = 'boys'
lno_cc.no_type = 'ie'
no_type = 'ie'
lo_type = 'boys'
frag_lolist = '1o'
# lno_cc.frag_lolist = '1o'
lno_cc.force_outcore_ao2mo = True

log = logger.new_logger(lno_cc)
_fdot = np.dot
fdot = lambda *args: reduce(_fdot, args)


s1e = lno_cc._scf.get_ovlp()

# NO type
if no_type == 'cim':
    no_type = 'ie'
elif no_type == 'edmet':
    no_type = 'rr'
log.info('no_type = %s', no_type)

# LO construction
orbloc = None
if orbloc is None:
    log.info('Constructing LOs')
    orbloc = lno_cc.get_lo(lo_type=lo_type)
else:
    log.info('Using user input LOs')

orbactocc = lno_cc.split_mo()[1]
m = fdot(orbloc.T, s1e, orbactocc)
lospanerr = abs(fdot(m.T, m) - np.eye(m.shape[1])).max()
if lospanerr > 1e-10:
    log.error('LOs do not fully span the occupied space! '
                'Max|<occ|LO><LO|occ>| = %e', lospanerr)
    raise RuntimeError

# check 2: Span(LO) == Span(occ)
occspanerr = abs(fdot(m, m.T) - np.eye(m.shape[0])).max()
if occspanerr < 1e-10:
    log.info('LOs span exactly the occupied space.')
    if no_type not in ['ir','ie']:
        log.error('"no_type" must be "ir" or "ie".')
        raise ValueError
else:
    log.info('LOs span occupied space plus some virtual space.')

# LO assignment to fragments
if frag_lolist is None:
    frag_atmlist = None
    if frag_atmlist is None:
        log.info('Grouping LOs by single-atom fragments')
        from ad_afqmc.lno.tools import autofrag
        frag_atmlist = autofrag(lno_cc.mol)
    else:
        log.info('Grouping LOs by user input atom-based fragments')
    frag_lolist = map_lo_to_frag(lno_cc.mol, orbloc, frag_atmlist,
                                    verbose=lno_cc.verbose)
elif frag_lolist == '1o':
    log.info('Using single-LO fragment')
    frag_lolist = [[i] for i in range(orbloc.shape[1])]
else:
    log.info('Using user input LO-fragment assignment')
nfrag = len(frag_lolist)

# frag weights
frag_wghtlist = None
if frag_wghtlist is None:
    frag_wghtlist = np.ones(nfrag)
elif isinstance(frag_wghtlist, (list,np.ndarray)):
    if len(frag_wghtlist) != nfrag:
        log.error('Input frag_wghtlist has wrong length (expecting %d; '
                    'got %d).', nfrag, len(frag_wghtlist))
        raise ValueError
    frag_wghtlist = np.asarray(frag_wghtlist)
else:
    log.error('Input frag_wghtlist has wrong data type (expecting '
                'array-like; got %s)', type(frag_wghtlist))
    raise ValueError

frag_nonvlist = None
if frag_nonvlist is None: frag_nonvlist = lno_cc.frag_nonvlist

# dump info
frag_atmlist = None
log.info('nfrag = %d  nlo = %d', nfrag, orbloc.shape[1])
log.info('frag_atmlist = %s', frag_atmlist)
log.info('frag_lolist = %s', frag_lolist)
log.info('frag_wghtlist = %s', frag_wghtlist)
log.info('frag_nonvlist = %s', frag_nonvlist)

# log.timer('LO and fragment  ', *cput0)

# frag_res = lno.kernel(lno_cc, orbloc, frag_lolist, no_type,
#                     eris=eris, frag_nonvlist=frag_nonvlist)

nfrag = len(frag_lolist)
if(type(lno_cc).__name__ =='LNOAFQMC'): 
    canonicalize = False;lno_cc.maxError = lno_cc.maxError/np.sqrt(nfrag)
else: canonicalize = True 
log = logger.Logger(lno_cc.stdout, lno_cc.verbose)
cput0 = (logger.process_clock(), logger.perf_counter())

# quick sanity check for no_type (more in 'make_fpno1')
if not (no_type[0] in 'rei' and no_type[1] in 'rei'):
    log.warn('Input no_type "%s" is invalid.', no_type)
    raise ValueError

if frag_nonvlist is None: frag_nonvlist = [[None,None]] * nfrag

if eris is None: eris = lno_cc.ao2mo()

cput1 = (logger.process_clock(), logger.perf_counter())
## Loop over fragment
frag_res = [None] * nfrag

for ifrag in [0]:
# local basis for internal space
    if(len(lno_cc.runfrags)>0):
        if(ifrag not in lno_cc.runfrags):frag_res[ifrag] = (0,0,0);continue
    fraglo = frag_lolist[ifrag]
    orbfragloc = orbloc[:,fraglo]
    frag_target_nocc, frag_target_nvir = frag_nonvlist[ifrag]
    THRESH_INTERNAL = 1e-10
    frozen_mask = lno_cc.get_frozen_mask()
    thresh_pno = [lno_cc.thresh_occ, lno_cc.thresh_vir]
    frzfrag, orbfrag, can_orbfrag = lno.make_fpno1(lno_cc, eris, orbfragloc, no_type,
                                               THRESH_INTERNAL, thresh_pno,
                                               frozen_mask=frozen_mask,
                                               frag_target_nocc=frag_target_nocc,
                                               frag_target_nvir=frag_target_nvir,
                                               canonicalize=canonicalize)
    
    frag_msg, frag_res = lno_cc.impurity_solve(
        mf,orbfrag,orbfragloc,frozen=frzfrag,eris=eris,log=log)
    print(frag_msg)

no_type = ie
Constructing LOs
lo_type = boys


******** <class 'pyscf.lo.boys.Boys'> ********
conv_tol = 1e-06
conv_tol_grad = None
max_cycle = 100
max_stepsize = 0.01
max_iters = 20
kf_interval = 5
kf_trust_region = 5
ah_start_tol = 1000000000.0
ah_start_cycle = 1
ah_level_shift = 0
ah_conv_tol = 1e-12
ah_lindep = 1e-14
ah_max_cycle = 40
ah_trust_region = 3
init_guess = atomic
Set conv_tol_grad to 0.000316228


macro= 1  f(x)= 17.85967232466  delta_f= 17.8597  |g|= 8.34017  1 KF 3 Hx
macro= 2  f(x)= 17.276939975683  delta_f= -0.582732  |g|= 7.45872  1 KF 3 Hx
macro= 3  f(x)= 16.767226643467  delta_f= -0.509713  |g|= 6.61597  1 KF 3 Hx
macro= 4  f(x)= 16.338077021812  delta_f= -0.42915  |g|= 5.67942  1 KF 3 Hx
macro= 5  f(x)= 15.995901310859  delta_f= -0.342176  |g|= 4.66179  1 KF 3 Hx
macro= 6  f(x)= 15.745867055193  delta_f= -0.250034  |g|= 3.57703  1 KF 3 Hx
macro= 7  f(x)= 15.591812901904  delta_f= -0.154054  |g|= 2.44004  1 KF 3 Hx
macro= 8  f(x)= 15.536189683087  delta_f= -0.0556232  |g|= 0.0719243  2 KF 3 Hx
macro= 9  f(x)= 15.536012700068  delta_f= -0.000176983  |g|= 0.0719243  1 KF 2 Hx
macro= 10  f(x)= 15.536012700068  delta_f= 2.84217e-14  |g|= 1.15904e-06  1 KF 1 Hx
macro X = 10  f(x)= 15.536012700068  |g|= 1.15904e-06  20 intor 11 KF 27 Hx
LOs span exactly the occupied space.
Using single-LO fragment
nfrag = 4  nlo = 4
frag_atmlist = None
frag_lolist = [[0], [1], [2], [3]]
frag_wg

In [109]:
print(orbfrag[:,:3])
print(orbfrag2[:,:3])

[[-0.0337372   0.1919195   0.37674517]
 [-0.02737632  0.31939606  0.41701319]
 [-0.00385525  0.39504568  0.05061981]
 [ 0.00149426  0.36684839 -0.26344249]
 [ 0.08227627  0.22267129 -0.41179183]
 [ 0.26874331  0.08232525 -0.23022784]
 [ 0.53081841 -0.06621269  0.16402082]
 [ 0.44489514 -0.08094387  0.23287274]]
[[-0.0337372   0.1919195   0.37674517]
 [-0.02737632  0.31939606  0.41701319]
 [-0.00385525  0.39504568  0.05061981]
 [ 0.00149426  0.36684839 -0.26344249]
 [ 0.08227627  0.22267129 -0.41179183]
 [ 0.26874331  0.08232525 -0.23022784]
 [ 0.53081841 -0.06621269  0.16402082]
 [ 0.44489514 -0.08094387  0.23287274]]


In [105]:
orbfragloc-orbfragloc1

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [114]:
print(eorb_cc3,eorb_cc4)

-0.03266823414308706 -0.03266818098290849


In [113]:
eorb_cc3,_,_ = lno_maker.lno_cc_solver(
                        mf,orbfrag1,orbfragloc1,frozen=frzfrag1)
_,(_,eorb_cc4,_) = lno_cc.impurity_solve(
         mf,orbfrag2,orbfragloc1,frozen=frzfrag1,eris=eris)
print(eorb_cc3)


WARN: CCSD detected DF being used in the HF object. MO integrals are computed based on the DF 3-index tensors.
It's recommended to use dfccsd.CCSD for the DF-CCSD calculations

Init t2, MP2 energy = -4.23090850510786  E_corr(MP2) -0.0564418088367092

******** <class 'pyscf.cc.dfccsd.RCCSD'> ********
CC2 = 0
CCSD nocc = 3, nmo = 7
frozen orbitals [0]
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 4000 MB (current use 275 MB)
Init E_corr(RCCSD) = -0.0564418088367276
cycle = 1  E_corr(RCCSD) = -0.0730404333151231  dE = -0.0165986245  norm(t1,t2) = 0.0522217
cycle = 2  E_corr(RCCSD) = -0.0817046377137844  dE = -0.0086642044  norm(t1,t2) = 0.0315309
cycle = 3  E_corr(RCCSD) = -0.0909941846782581  dE = -0.00928954696  norm(t1,t2) = 0.0199549
cycle = 4  E_corr(RCCSD) = -0.0942635457138072  dE = -0.00326936104  norm(t1,t2) = 0.0102902
cycle = 5  E_corr(RCCSD) = -0.0953669513647346  dE = -0.0011034